In [1]:
data_samples <- read.csv("data_melatonion.csv", check.names = TRUE)
data_samples_melatonin_for_statistics <- data_samples
#data_samples <- read.csv("filter_samples_Day1_2.csv", check.names = TRUE)
#data_zotus<-read.csv("data_zotus/Final_ZOTUs_results3.csv", check.names = FALSE)
data_zotus<-read.csv("Final_ZOTUs_results_Aug_24.csv", check.names = FALSE)
#data_samples
data_clock_ctool<-read.csv("Melatonin-Clock-Stool.csv", check.names = FALSE)


In [2]:
# Perform the merge to add Clock_stool from df_B to df_A
data_samples <- merge(data_samples, data_clock_ctool[, c("Cohort", "Sample_id", "Timepoint", "Day", "Clock_stool")],
                   by = c("Cohort", "Sample_id", "Timepoint", "Day"),
                   all.x = TRUE)  # Keep all rows from df_A, even if no match in df_B


In [3]:
# Convert time string to numeric hours
data_samples$Clock_stool_numeric <- sapply(strsplit(data_samples$Clock_stool, ":"), function(x) {
  as.numeric(x[1]) + as.numeric(x[2]) / 60
})



In [4]:
Study_variable <- 'Melatonin_pg_g_Stool'  # 'Time_since_last_stool' , 'Clock_stool' , 'Time_since_last_meal', 'Time_since_last_sleep'
as.symbol(Study_variable)
Age <- 'all'     #'all' , '3 months', '6 months';, '12 months'



Melatonin_pg_g_Stool

In [5]:
library(dplyr)
#data_samples<-data_samples[,c('Cohort', 'Sample_Code_16s',  'Timepoint', 'Clock_stool',	'Time_since_last_stool',	'Time_since_last_meal',	'Time_since_last_sleep')]
data_samples<-data_samples[,c('Cohort', 'Sample_Code_16s',  'Timepoint', 'Clock_stool_numeric', Study_variable)]



Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [6]:
# Function to extract age group based on conditions
 
extract_age_group <- function(sample_id) {

      # Check if the sample ID is NA
  if (is.na(sample_id)) {
    return(NA)
  }

    
  # Check if the sample ID starts with 'S'
  if (startsWith(sample_id, 'S')) {
    # Split the sample ID by '-'
    parts <- strsplit(sample_id, "-")[[1]]
    # Get the second division
    division <- parts[2]
    # Extract the first two characters
    return(as.integer(substr(division, 1, 2)))
  }
    
  # Split the sample ID by '-'
  parts <- strsplit(sample_id, "-")[[1]]
  # Get the second division
  division <- parts[2]
  # Extract the last character
  last_char <- substring(division, nchar(division), nchar(division))
  # Check if the sample ID starts with 'I'
  if (startsWith(sample_id, 'I')) {
    # If so, multiply the last character by 3
    return(as.integer(last_char) * 3)
    #return(0)       # Sleepy cohort ages between 5 and 31 months, so, will be excluded
  } else {
    # Otherwise, return the last character as integer
    return(as.integer(last_char))
  }
}

# Apply the function to create the age group column
data_samples$age_group <- sapply(data_samples$Sample_Code_16s, extract_age_group)

# View the updated data
#print(data)


dim(data_samples)
data_samples[1:3,]

[1] 1115    6

Cohort Sample_Code_16s Timepoint Clock_stool_numeric Melatonin_pg_g_Stool
1 COLIC  C03-03          t1        NA                                      
2 COLIC  C04-03          t1        NA                                      
3 COLIC  C05-03          t1        NA                                      
  age_group
1 3        
2 3        
3 3

In [7]:
data_samples <- data_samples[data_samples$Melatonin_pg_g_Stool!='nan',]
data_samples <- data_samples[data_samples$Melatonin_pg_g_Stool!='',]

# Z-score Normalization
data_samples <- data_samples %>%
  mutate(Melatonin_pg_g_Stool_scaled = (as.numeric(Melatonin_pg_g_Stool) - mean(as.numeric(Melatonin_pg_g_Stool), na.rm = TRUE)) / 
          sd(as.numeric(Melatonin_pg_g_Stool), na.rm = TRUE))

data_samples <- data_samples %>%
  mutate(Melatonin_pg_g_Stool = as.numeric(Melatonin_pg_g_Stool))


data_samples <- data_samples[data_samples$Melatonin_pg_g_Stool < 1000 & data_samples$Melatonin_pg_g_Stool > 100, ]



Warning message:
"There were 3 warnings in `mutate()`.
The first warning was:
ℹ In argument: `Melatonin_pg_g_Stool_scaled = `/`(...)`.
Caused by warning:
! NAs introduced by coercion
ℹ Run `dplyr::last_dplyr_warnings()` to see the 2 remaining warnings."
Warning message:
"There was 1 warning in `mutate()`.
ℹ In argument: `Melatonin_pg_g_Stool = as.numeric(Melatonin_pg_g_Stool)`.
Caused by warning:
! NAs introduced by coercion"


In [8]:

data_samples_clean <- na.omit(data_samples)
 

In [9]:

# Merge df1 with the filtered df2
merged_data <- merge(data_samples_clean, data_zotus, by = "Sample_Code_16s", all.x = FALSE)

dim(merged_data)

# Get the dimensions of merged_data
data_size <- dim(merged_data)


[1] 396  17

In [10]:
merged_data_age <- merged_data[merged_data$Cohort != 'SLEEPY', ]  # exclude sleepy cohort from age comparison
#merged_data_age <- merged_data
if (Age == '3 months'){
    merged_data <- merged_data_age[merged_data_age$age_group ==3,]}

if (Age == '6 months'){
    merged_data <- merged_data_age[merged_data_age$age_group ==6,]}

if (Age == '12 months'){
    merged_data <- merged_data_age[merged_data_age$age_group ==12,]}



In [11]:
# Check for duplicates
if (any(duplicated(merged_data$Sample_Code_16s))) {
  # Remove duplicates, keeping only the first occurrence
  merged_data <- merged_data[!duplicated(merged_data$Sample_Code_16s), ]
  
  # Print a message indicating duplicates were removed
  message("Duplicates were found and removed.")
} else {
  # Print a message indicating no duplicates were found
  message("No duplicates were found.")
}

# View the cleaned dataframe
head(merged_data)
dim(merged_data)

No duplicates were found.



Sample_Code_16s Cohort Timepoint Clock_stool_numeric Melatonin_pg_g_Stool
1 F300-M6-2       NUTR   t2         7.250000           155.03              
2 F302-M3-2       NUTR   t1        12.000000           119.22              
3 F302-M6-1       NUTR   t2         9.833333           307.46              
4 F302-M6-2       NUTR   t2         8.000000           435.98              
5 F306-M6-1       NUTR   t2        18.000000           334.52              
6 F306-M6-2       NUTR   t2         9.000000           182.20              
  age_group Melatonin_pg_g_Stool_scaled Abundance_all Shannon_Diversity
1 6         -0.21522676                  66615        3.481963         
2 3         -0.47815104                  76856        2.386428         
3 6          0.90394522                  78310        2.515107         
4 6          1.84756513                  63635        2.458420         
5 6          1.10262523                 101882        2.141278         
6 6         -0.01573911                  73329        2.640028         
  Simpson_Diversity Richness Evenness  Rel_Abun_0.1 Rel_Abun_0.01
1 0.9159997         607      0.5405561 2            17           
2 0.8358981         247      0.4333782 3             9           
3 0.8581872         250      0.4428137 2            12           
4 0.8524057         214      0.4566770 3            15           
5 0.8095004         298      0.3617230 4             8           
6 0.8791688         355      0.4388114 4            11           
  Rel_Abun_0.001 Rel_Abun_0.0001 Rel_Abun_0.00001
1 69             216             607             
2 41              92             247             
3 30              88             250             
4 23              85             214             
5 27              81             191             
6 34             111             355

[1] 396  17

In [12]:
merged_data$baby_id <- sub("-.*", "", merged_data$Sample_Code_16s)
merged_data

Sample_Code_16s Cohort Timepoint Clock_stool_numeric Melatonin_pg_g_Stool
1   F300-M6-2       NUTR   t2         7.250000           155.03              
2   F302-M3-2       NUTR   t1        12.000000           119.22              
3   F302-M6-1       NUTR   t2         9.833333           307.46              
4   F302-M6-2       NUTR   t2         8.000000           435.98              
5   F306-M6-1       NUTR   t2        18.000000           334.52              
6   F306-M6-2       NUTR   t2         9.000000           182.20              
7   F309-M3-1       NUTR   t1        20.666667           144.26              
8   F309-M3-2       NUTR   t1        12.000000           128.02              
9   F309-M6-1       NUTR   t2         8.333333           258.63              
10  F309-M6-2       NUTR   t2        18.750000           145.35              
11  F310-M3-1       NUTR   t1         7.333333           316.66              
12  I001-T1-1       SLEEPY pre       12.416667           210.96              
13  I001-T1-2       SLEEPY pre        8.166667           335.18              
14  I002-T1-1       SLEEPY pre        6.666667           300.71              
15  I002-T1-2       SLEEPY pre        7.000000           360.80              
16  I002-T2-1       SLEEPY post      12.750000           307.20              
17  I002-T2-2       SLEEPY post       8.750000           429.96              
18  I003-T1-1       SLEEPY pre       11.250000           115.83              
19  I003-T2-1       SLEEPY post       9.500000           150.18              
20  I003-T2-2       SLEEPY post      13.500000           161.86              
21  I004-T1-1       SLEEPY pre        8.833333           135.60              
22  I005-T1-1       SLEEPY pre        7.166667           124.57              
23  I005-T1-2       SLEEPY pre        7.000000           140.80              
24  I006-T1-1       SLEEPY pre        9.250000           158.80              
25  I006-T1-2       SLEEPY pre       12.750000           115.47              
26  I006-T2-1       SLEEPY post      15.500000           200.14              
27  I006-T2-2       SLEEPY post      15.500000           162.88              
28  I007-T1-1       SLEEPY pre        8.000000           129.33              
29  I007-T2-1       SLEEPY post      14.333333           104.41              
30  I007-T2-2       SLEEPY post       7.000000           109.18              
⋮   ⋮               ⋮      ⋮         ⋮                   ⋮                   
367 S148-12         SDEGU  t3        17.500000           283.28              
368 S149-03         SDEGU  t1        17.666667           102.11              
369 S149-06         SDEGU  t2         9.166667           487.43              
370 S149-12         SDEGU  t3         8.200000           321.06              
371 S150-03         SDEGU  t1        15.500000           234.30              
372 S150-06         SDEGU  t2         6.916667           114.19              
373 S150-12         SDEGU  t3         8.750000           138.78              
374 S152-03         SDEGU  t1        18.000000           303.72              
375 S152-12         SDEGU  t3         8.500000           151.31              
376 S153-03         SDEGU  t1        10.000000           114.75              
377 S153-06         SDEGU  t2        11.500000           172.00              
378 S153-12         SDEGU  t3        17.000000           311.81              
379 S154-03         SDEGU  t1         7.250000           506.22              
380 S154-06         SDEGU  t2         7.916667           155.86              
381 S154-12         SDEGU  t3         8.166667           149.55              
382 S156-03         SDEGU  t1         3.383333           207.47              
383 S156-06         SDEGU  t2         8.266667           166.38              
384 S156-12         SDEGU  t3         8.450000           115.19              
385 S157-03         SDEGU  t1         7.833333           534.24              
386 S157-06         SDEGU 

In [13]:
# Load necessary libraries
library(Maaslin2)

# Create input_data as a data frame and set row names
input_data1 <- merged_data[, c('Shannon_Diversity', 'Richness','Evenness',	'Rel_Abun_0.1'), drop = FALSE]
rownames(input_data1) <- merged_data$Sample_Code_16s

# Create input_metadata as a data frame and set row names
input_metadata1 <- merged_data[, c('Melatonin_pg_g_Stool','Clock_stool_numeric' ), drop = FALSE]
rownames(input_metadata1) <- merged_data$Sample_Code_16s

# Check and clean missing data
input_data1 <- na.omit(input_data1)
input_metadata1 <- na.omit(input_metadata1)

# Ensure output directory exists
dir.create("results", showWarnings = FALSE)

# Run MaAslin2 for multivariate testing
maaslin_results <- Maaslin2(
    input_data = input_data1,
    input_metadata = input_metadata1,
    output = paste0("results/", Age,"/"),
    fixed_effects = c('Melatonin_pg_g_Stool','Clock_stool_numeric'), # Include independent variables and covariate
    random_effects = NULL, # Use random effects if needed
    normalization = 'NONE', # Or another method if required
    standardize = TRUE
)


Warning message:
"package 'Maaslin2' was built under R version 4.4.2"


[1] "Creating output folder"
[1] "Creating output feature tables folder"
[1] "Creating output fits folder"
[1] "Creating output figures folder"
2025-03-26 16:38:31.75662 INFO::Writing function arguments to log file
2025-03-26 16:38:31.78321 INFO::Verifying options selected are valid
2025-03-26 16:38:31.807509 INFO::Determining format of input files
2025-03-26 16:38:31.809346 INFO::Input format is data samples as rows and metadata samples as rows
2025-03-26 16:38:31.812138 INFO::Formula for fixed effects: expr ~  Melatonin_pg_g_Stool + Clock_stool_numeric
2025-03-26 16:38:31.813317 INFO::Filter data based on min abundance and min prevalence
2025-03-26 16:38:31.81427 INFO::Total samples in data: 396
2025-03-26 16:38:31.815228 INFO::Min samples required with min abundance for a feature not to be filtered: 39.600000
2025-03-26 16:38:31.816795 INFO::Total filtered features: 0
2025-03-26 16:38:31.817902 INFO::Filtered feature names from abundance and prevalence filtering:
2025-03-26 16:38:31

`geom_smooth()` using formula = 'y ~ x'


2025-03-26 16:38:32.869822 INFO::Creating scatter plot for continuous data, Melatonin_pg_g_Stool vs Richness


`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'


In [14]:
# Load necessary libraries
library(Maaslin2)

# Create input_data as a data frame and set row names
input_data1 <- merged_data[, c('Shannon_Diversity', 'Richness','Evenness',	'Rel_Abun_0.1'), drop = FALSE]
rownames(input_data1) <- merged_data$Sample_Code_16s

# Create input_metadata as a data frame and set row names
input_metadata1 <- merged_data[, c('Melatonin_pg_g_Stool','Clock_stool_numeric', 'age_group', 'baby_id' ), drop = FALSE]
rownames(input_metadata1) <- merged_data$Sample_Code_16s

# Check and clean missing data
input_data1 <- na.omit(input_data1)
input_metadata1 <- na.omit(input_metadata1)

# Ensure output directory exists
output_dir <- paste0("results/random_effects/")
dir.create(output_dir, recursive = TRUE, showWarnings = FALSE)

# Run MaAslin2 for multivariate testing
maaslin_results <- Maaslin2(
    input_data = input_data1,
    input_metadata = input_metadata1,
    output = output_dir,
    fixed_effects = c('Melatonin_pg_g_Stool','Clock_stool_numeric'), # Include independent variables and covariate
    random_effects = c('baby_id'), # Use random effects if needed
    normalization = 'NONE', # Or another method if required
    standardize = TRUE
)


[1] "Creating output feature tables folder"
[1] "Creating output fits folder"
[1] "Creating output figures folder"
2025-03-26 16:38:33.465381 INFO::Writing function arguments to log file
2025-03-26 16:38:33.473725 INFO::Verifying options selected are valid
2025-03-26 16:38:33.474727 INFO::Determining format of input files
2025-03-26 16:38:33.475765 INFO::Input format is data samples as rows and metadata samples as rows
2025-03-26 16:38:33.478125 INFO::Formula for random effects: expr ~ (1 | baby_id)
2025-03-26 16:38:33.479101 INFO::Formula for fixed effects: expr ~  Melatonin_pg_g_Stool + Clock_stool_numeric
2025-03-26 16:38:33.480051 INFO::Filter data based on min abundance and min prevalence
2025-03-26 16:38:33.480936 INFO::Total samples in data: 396
2025-03-26 16:38:33.48182 INFO::Min samples required with min abundance for a feature not to be filtered: 39.600000
2025-03-26 16:38:33.482932 INFO::Total filtered features: 0
2025-03-26 16:38:33.483918 INFO::Filtered feature names from 

boundary (singular) fit: see help('isSingular')



2025-03-26 16:38:33.818161 INFO::Fitting model to feature number 2, Richness


boundary (singular) fit: see help('isSingular')



2025-03-26 16:38:33.87691 INFO::Fitting model to feature number 3, Evenness
2025-03-26 16:38:33.920069 INFO::Fitting model to feature number 4, Rel_Abun_0.1
2025-03-26 16:38:33.964691 INFO::Counting total values for each feature
2025-03-26 16:38:33.966637 INFO::Writing filtered data to file results/random_effects//features/filtered_data.tsv
2025-03-26 16:38:33.971213 INFO::Writing filtered, normalized data to file results/random_effects//features/filtered_data_norm.tsv
2025-03-26 16:38:33.975152 INFO::Writing filtered, normalized, transformed data to file results/random_effects//features/filtered_data_norm_transformed.tsv
2025-03-26 16:38:33.980143 INFO::Writing residuals to file results/random_effects//fits/residuals.rds
2025-03-26 16:38:33.9819 INFO::Writing fitted values to file results/random_effects//fits/fitted.rds
2025-03-26 16:38:33.983842 INFO::Writing extracted random effects to file results/random_effects//fits/ranef.rds
2025-03-26 16:38:33.985349 INFO::Writing all results t

`geom_smooth()` using formula = 'y ~ x'


2025-03-26 16:38:34.13326 INFO::Creating scatter plot for continuous data, Melatonin_pg_g_Stool vs Richness


`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'


In [15]:
maaslin_results

$results
            feature             metadata                value         coef
1 Shannon_Diversity Melatonin_pg_g_Stool Melatonin_pg_g_Stool -0.106680768
2          Richness Melatonin_pg_g_Stool Melatonin_pg_g_Stool -0.123382288
3 Shannon_Diversity  Clock_stool_numeric  Clock_stool_numeric  0.009849884
4          Richness  Clock_stool_numeric  Clock_stool_numeric -0.018613230
5          Evenness Melatonin_pg_g_Stool Melatonin_pg_g_Stool -0.017873272
6          Evenness  Clock_stool_numeric  Clock_stool_numeric -0.010450882
7      Rel_Abun_0.1 Melatonin_pg_g_Stool Melatonin_pg_g_Stool  0.014107217
8      Rel_Abun_0.1  Clock_stool_numeric  Clock_stool_numeric  0.026096565
      stderr         pval                 name        qval   N N.not.zero
1 0.03161975 0.0008148948 Melatonin_pg_g_Stool 0.006519158 396        396
2 0.04094315 0.0027497882 Melatonin_pg_g_Stool 0.010999153 396        396
3 0.03161975 0.7555777516  Clock_stool_numeric 0.755577752 396        396
4 0.04094315 0.6496396835  Clock_stool_numeric 0.755577752 396        396
5 0.02227785 0.4228870324 Melatonin_pg_g_Stool 0.755577752 396        396
6 0.02232583 0.6399855568  Clock_stool_numeric 0.755577752 396        396
7 0.03334971 0.6725203693 Melatonin_pg_g_Stool 0.755577752 396        392
8 0.03359612 0.4377678474  Clock_stool_numeric 0.755577752 396        392

$residuals
                    F300-M6-2     F302-M3-2  F302-M6-1  F302-M6-2    F306-M6-1
Shannon_Diversity  0.58272370 -0.0007490456  0.2252171  0.2956808 -0.004592671
Richness           0.73162367 -0.5771408222 -0.4009993 -0.5184676 -0.088413500
Evenness           0.38251181  0.0712101723  0.1214024  0.1780771 -0.124564339
Rel_Abun_0.1      -0.01484389  0.4265385402 -0.1645985  0.4182787  0.661778713
                    F306-M6-2   F309-M3-1   F309-M3-2   F309-M6-1   F309-M6-2
Shannon_Diversity  0.20037117 -0.19259563 -0.07313323 -0.06221546 -0.15528608
Richness          -0.01049165 -0.58332071 -0.87379699 -0.39999827 -0.19244557
Evenness           0.11276938 -0.05438322  0.07983766 -0.07846777 -0.09700395
Rel_Abun_0.1       0.73147685  0.23489735  0.70373933  0.29743433  0.24631863
                    F310-M3-1  I001-T1-1  I001-T1-2 I002-T1-1  I002-T1-2
Shannon_Diversity -0.33341438  0.7699704  0.8970785 0.3602931 0.42780336
Richness          -0.14048190 -0.7275873  1.4884346 0.1094564 0.09912076
Evenness           0.22974482  0.7764336  0.4530558 0.1659403 0.20746714
Rel_Abun_0.1      -0.02936241 -0.7875049 -0.7746104 0.3549816 0.34684334
                   I002-T2-1 I002-T2-2  I003-T1-1  I003-T2-1  I003-T2-2
Shannon_Diversity  0.3509022 0.4747225  0.2586333  0.5870386  0.5706874
Richness          -0.3488740 0.2257507  0.3074511  0.7298026  0.6170218
Evenness           0.2669045 0.2084535  0.1429951  0.3777601  0.3943697
Rel_Abun_0.1      -0.2672465 0.7442928 -0.6848162 -0.6777922 -0.7030521
                    I004-T1-1  I005-T1-1  I005-T1-2  I006-T1-1  I006-T1-2
Shannon_Diversity  0.33652304  0.4265068  0.4298011  0.7279328  0.6983716
Richness           0.26342531  0.6384339  0.6195013  0.8967629  1.3276729
Evenness           0.24509158  0.4229446  0.3494492  0.4282599  0.3747526
Rel_Abun_0.1      -0.02128775 -0.7556591 -0.7563127 -0.5007105 -0.5173477
                   I006-T2-1  I006-T2-2   I007-T1-1   I007-T2-1   I007-T2-2
Shannon_Diversity  0.8635308  0.8770650 -0.24144263 0.023240276  0.02882332
Richness           1.6278308  1.3196462  0.12456133 0.278641138  0.19895990
Evenness           0.4684070  0.5477772 -0.27914123 0.006483072 -0.01718822
Rel_Abun_0.1      -0.5425356 -1.5387330 -0.07567719 0.473721111 -0.06760376
                    I008-T1-1  I009-T2-1  I009-T2-2  I010-T1-1  I010-T2-1
Shannon_Diversity -0.21137625  0.3330868 0.09900267 0.18374869  0.2414398
Richness          -0.24677596  0.8277413 0.19798680 0.33430879  0.4796297
Evenness          -0.16896532  0.1717627 0.02208786 0.06558127  0.1478015
Rel_Abun_0.1      -0.03357585 -0.1081401 0.43235791 0.51783182 -0.1004943
        

In [17]:
# Compute Pearson correlation matrix

# Select relevant columns
corr_data <- merged_data[, c("Melatonin_pg_g_Stool_scaled", "Richness", "Shannon_Diversity", "Evenness", "Rel_Abun_0.1")]

# Compute Pearson correlation matrix
pearson_corr <- cor(corr_data, method = "pearson", use = "complete.obs")

# Define the output file path
output_file <- paste0("results/", Age, "/pearson_correlation.txt")

# Save results to a text file
write.table(pearson_corr, file = output_file, sep = "\t", quote = FALSE, row.names = TRUE, col.names = NA)

# Print correlation matrix
pearson_corr


Melatonin_pg_g_Stool_scaled Richness  
Melatonin_pg_g_Stool_scaled  1.00000000                 -0.1484802
Richness                    -0.14848018                  1.0000000
Shannon_Diversity           -0.14791699                  0.8205909
Evenness                    -0.04845864                  0.6329334
Rel_Abun_0.1                 0.02963052                 -0.2609947
                            Shannon_Diversity Evenness    Rel_Abun_0.1
Melatonin_pg_g_Stool_scaled -0.14791699       -0.04845864  0.02963052 
Richness                     0.82059093        0.63293338 -0.26099474 
Shannon_Diversity            1.00000000        0.85841940 -0.08889966 
Evenness                     0.85841940        1.00000000 -0.11932397 
Rel_Abun_0.1                -0.08889966       -0.11932397  1.00000000

In [18]:
# Preprocess data to remove NaN, blank, or non-numeric values for Melatonin_pg_g_Stool
data_samples_melatonin_for_statistics <- data_samples_melatonin_for_statistics %>% 
  filter(!is.na(Melatonin_pg_g_Stool) & Melatonin_pg_g_Stool != "" & Melatonin_pg_g_Stool != "nan" & Melatonin_pg_g_Stool != "NA") #& is.numeric(Melatonin_pg_g_Stool)

library(dplyr)

# Convert Melatonin_pg_g_Stool to numeric and handle potential NA or non-numeric values
data_samples_melatonin_for_statistics2 <- data_samples_melatonin_for_statistics %>%
  mutate(Melatonin_pg_g_Stool = as.numeric(Melatonin_pg_g_Stool))

# Check for NA values after conversion
if (any(is.na(data_samples_melatonin_for_statistics2$Melatonin_pg_g_Stool))) {
  warning("NA values were introduced during conversion. Check for non-numeric values in the original column.")
}

# View the updated dataframe
data_samples_melatonin_for_statistics2

Warning message:
"There was 1 warning in `mutate()`.
ℹ In argument: `Melatonin_pg_g_Stool = as.numeric(Melatonin_pg_g_Stool)`.
Caused by warning:
! NAs introduced by coercion"
Warning message:
"NA values were introduced during conversion. Check for non-numeric values in the original column."


Cohort Sample_id Timepoint Day Sample_Code_16s Melatonin_pg_g_Stool
1   SPIN   M001      t2        1   M001-M6-1       145.53              
2   SPIN   M001      t2        2   M001-M6-2       176.03              
3   SPIN   M002      t2        1   M002-M6-1       275.78              
4   SPIN   M002      t2        2   M002-M6-2       185.76              
5   SPIN   M003      t1        1   M003-M3-1        83.76              
6   SPIN   M003      t1        2   M003-M3-2       110.62              
7   SPIN   M003      t2        1   M003-M6-1       144.24              
8   SPIN   M003      t2        2   M003-M6-2       324.54              
9   SPIN   M004      t1        1   M004-M3-1       101.31              
10  SPIN   M004      t1        2   M004-M3-2       148.01              
11  SPIN   M004      t2        1   M004-M6-1        86.98              
12  SPIN   M004      t2        2   M004-M6-2       110.18              
13  SPIN   M005      t1        1   M005-M3-1       834.76              
14  SPIN   M005      t1        2   M005-M3-2       995.73              
15  SPIN   M005      t2        1   M005-M6-1       142.16              
16  SPIN   M005      t2        2   M005-M6-2       110.83              
17  SPIN   M006      t1        1   M006-M3-1       212.31              
18  SPIN   M006      t1        2   M006-M3-2       267.26              
19  SPIN   M006      t2        1   M006-M6-1       345.65              
20  SPIN   M006      t2        2   M006-M6-2       291.66              
21  SPIN   M007      t1        1   M007-M3-1       163.80              
22  SPIN   M007      t1        2   M007-M3-2       187.09              
23  SPIN   M007      t2        1   M007-M6-1       117.77              
24  SPIN   M007      t2        2   M007-M6-2       172.45              
25  SPIN   M009      t1        1   M009-M3-1       229.48              
26  SPIN   M009      t1        2   M009-M3-2       993.11              
27  SPIN   M009      t2        1   M009-M6-1       243.71              
28  SPIN   M009      t2        2   M009-M6-2       188.18              
29  SPIN   M010      t1        1   M010-M3-1       441.99              
30  SPIN   M010      t1        2   M010-M3-2       228.45              
⋮   ⋮      ⋮         ⋮         ⋮   ⋮               ⋮                   
964 FETAL  P007      nb        s03 P007_S03        103.66              
965 FETAL  P007      nb        s04 P007_S04        115.75              
966 FETAL  P007      nb        s05 P007_S05        153.05              
967 FETAL  P007      nb        s06 P007_S06        164.89              
968 FETAL  P007      nb        s07 P007_S07        104.78              
969 FETAL  P008      nb        s01 P008_S01         42.26              
970 FETAL  P008      nb        s02 P008_S02         37.82              
971 FETAL  P008      nb        s03 P008_S03         37.18              
972 FETAL  P008      nb        s04 P008_S04         51.85              
973 FETAL  P008      nb        s05 P008_S05         70.63              
974 FETAL  P008      nb        s06 P008_S06         53.46              
975 FETAL  P013      nb        s01 P013_S01        231.68              
976 FETAL  P013      nb        s02 P013_S02        141.05              
977 FETAL  P013      nb        s03 P013_S03        319.52              
978 FETAL  P021      nb        s01 P021_S01        249.43              
979 FETAL  P021      nb        s02 P021_S02        143.17              
980 FETAL  P021      nb        s03 P021_S03        137.89              
981 FETAL  P022      nb        s01 P022_S01        384.42              
982 FETAL  P022      nb        s02 P022_S02         94.08              
983 FETAL  P028      nb        s01 P028_S01         63.15              
984 FETAL  P028      nb        s02 P028_S02        100.06              
985 FETAL  P028      nb        s03 P028_S03         56.57              
986 FETAL  P028      nb        s04 P028_S04         62.98              
987 FETAL  P034      nb        s01 P034_S01 

In [19]:
# Calculate summary statistics for Melatonin_pg_g_Stool
melatonin_stats <- data_samples_melatonin_for_statistics2 %>% 
  summarise(
    Count = n(),
    Average = mean(Melatonin_pg_g_Stool, na.rm = TRUE),
    Max = max(Melatonin_pg_g_Stool, na.rm = TRUE),
    Min = min(Melatonin_pg_g_Stool, na.rm = TRUE),
    Std_Dev = sd(Melatonin_pg_g_Stool, na.rm = TRUE)
  )

# Print the results
print(melatonin_stats)

  Count  Average     Max   Min  Std_Dev
1   993 184.3436 1128.64 12.05 136.1989
